In [1]:
!pip install OpenAI
!pip install --upgrade gradio

In [2]:
import gradio as gr

In [3]:
def check_answer_correctness(answer,question):
    # Use GPT-4 to check the correctness of the answer
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
      {
        "role": "system",
        "content": f"As a friendly chatbot designed to help school-age children, whose age are between 8(eight) and 17(sixteen) "
        f"I want to ensure accurate information. Can you confirm if the response to the following question is correct?\n\n"
        f"Question: \"{question}\"\n\n"  f"Answer: \"{answer}\"\n\n"
        "for example for the age of child, their parents should enter a digit between 8, and 17, for example if they enter 12, this answer is correct,otherwise, the answer is incorrect if the anwser is not digit or is a number out of thiis range, another example is 'What is your child's gender?' the answer to this question can be female, male , girl, boy, man , woman, and so on , and it cannot be yellow for instance or for the traits, it should say, she/he is generous, kiind and sth like this. They should not say, she is upset for example\n"
        "Please respond with '1' if the answer is correct and matches the question or '0' if it needs correction without any explanation"
     },
      {
        "role": "user",
        "content": answer
      }
      ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # Extract the generated response
    generated_response = response.choices[0].message.content
    return generated_response.lower()

In [4]:
def question(number):
 questions = [
        "What is your child's name?",
        "How old is your child?",
        "What is your child's gender?",
        "How would you describe your child's personality traits?",
              ]
 return questions[number]

In [5]:
global counter

In [6]:
def counter(counter):
      counter = counter + 1
      return counter

In [7]:
chatHistory = []

In [26]:
questions = [
    "What is your child's name?",
    "How old is your child?",
    "What is your child's gender?",
    "How would you describe your child's personality traits?",
    "Do you want to edit some information?"
]

from openai import OpenAI
import time
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def check_answer_correctness2(answer):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "As a friendly chatbot designed to help school-age children, whose age ranges between 8 and 17, "
                "The system should determine the index of the question based on parents as a digit \n\n"
                "Question: \"Enter the index of the part you would like to edit from this category:{ '1' or '2', or '3', or '4'}\"\n\n"
                f"Answer: {answer}\\n\n"
                "For example, if the parent writes 'child name' or 'What is child's name' or '1', the prompt should be just '1'.\n "
                "If the parent writes an expression about the child's age or  enter '2', the prompt should be just '2'. \n"
                "If the parent answers 'What is your child's gender?' or anything related to this concept or '3', the prompt should be just'3'. \n"
                "If the parent asks about 'How would you describe your child's personality traits?' or anything related to characteristics of their child or '4', Note that the prompt should be just '4'.\n"
            },
            {
                "role": "user",
                "content": answer
            }
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    generated_response = response.choices[0].message.content
    part_to_edit_index = int(generated_response)
    return part_to_edit_index


def check_answer_correctness(question, answer):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": f"As a friendly chatbot designed to help school-age children, whose age are between 8(eight) and 18(eighteen) "
                            f"I want to ensure accurate information. Can you confirm if the response to the following question is correct?\n\n"
                            f"Question: \"{question}\"\n\n" f"Answer: \"{answer}\"\n\n"
                            "for example for the age of child, their parents should enter a digit between 8, and 18, otherwise, the answer is incorrect, another example is 'What is your child's gender?' the answer to this question can be female, male , girl, boy, man , woman, and so on , and it cannot be yellow for instance\n"
                            "for the questions 'How would you describe your child s personality traits?' the answer should be sth like this, like She is generous, but if the user said she is upset, this answer cannot be a correct answer since 'upset' is not a trait  or if it asks Do you want to edit information and the answer is yes return '1', and otherwise return '0'."
                            f"Please respond with '1' if the answer is correct or '0' if it needs correction without any explanation"
            },
            {
                "role": "user",
                "content": answer
            }
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    generated_response = response.choices[0].message.content
    return generated_response.lower()

chatHistory = []
counter = 0
index = 0
List = []
j = 0
edit_mode = False
counter0=0
flag=0
counter1=0
part_to_edit=''
async def manageConversation(sentence, history):
    global counter
    global index
    global List
    global j
    global edit_mode
    global counter0
    global part_to_edit
    chatHistory.append({
        "role": "user",
        "content": sentence
    })

    while chatHistory[index]['content'] != "1234":
        index = index + 1
        return "Your Password is incorrect..., Enter your password again!"

    if chatHistory[index]['content'] == "1234":
        if not edit_mode:
            if counter == 0:
                text = questions[counter]
                counter += 1
                return text
            elif 1 <= counter <= 4:
                if check_answer_correctness(questions[counter - 1], chatHistory[counter + index + j]['content']) == '1':
                    List.append(chatHistory[counter + index + j]['content'])
                    text = questions[counter]
                    counter += 1
                    return text
                elif check_answer_correctness(questions[counter - 1], chatHistory[counter + index + j]['content']) == '0':
                    j = j + 1
                    return "Your answer seems incorrect. Please try again and answer correctly..." + questions[counter - 1]
            elif counter > 4 and sentence.lower() == "yes":
                edit_mode = True
                return "Enter the the part you would like to edit from the information"
            elif counter > 4 and sentence.lower() == "no":
                return "Thank you"

    if edit_mode:
        if part_to_edit=='':
           counter += 1
        # Ask which part to edit
           part_to_edit = check_answer_correctness2(chatHistory[counter + index + j]['content'])

        # Get the updated information
           text=questions[int(part_to_edit)-1]
           return text
        # Check correctness using GPT-4
        else:
           counter += 1
           check = check_answer_correctness( questions[int(part_to_edit)-1],chatHistory[counter + index + j]['content'])
           if check == '0' and chatHistory[counter + index + j]['content'].lower()!='no' and chatHistory[counter + index + j]['content'].lower()!='yes':
              return "Your answer seems incorrect!! Enter the answer again"
           elif check=='1':
              List[int(part_to_edit)-1]=chatHistory[counter + index + j]['content']
        # Update the collected information
              return "Thanks. Would you like to edit any part? (yes/no): "
           elif chatHistory[counter + index + j]['content'].lower()=='yes':
              part_to_edit=''
              return "Enter the the part you would like to edit from the information"
           elif chatHistory[counter + index + j]['content'].lower()=='no':
              return "Thank you"
iface = gr.ChatInterface(
    manageConversation,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="How can I help you, " + '?', container=False, scale=7),
    title="WiseOldGPT",
    description="Ask me any question",
    theme="soft",
    examples=["Can you help me with my science project?", "I don't know why Mom doesn't buy me the car!", "Today was amazing!"],
    cache_examples=False,
    retry_btn="Retry",
    undo_btn="Delete Previous",
    clear_btn="Clear",
)

iface.launch(debug=True)



Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b13283557125b9bd4a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b13283557125b9bd4a.gradio.live


In [27]:
List

['Zahra', '12', 'female', 'She is so kind and honest']

In [10]:
if 0 <= counter <= 3 and counter0 > 4 and sentence.lower() != "yes" and sentence.lower() != "no":
  print('1')
else:
  print('0')

0


In [11]:
sentence='female'

In [12]:
check_answer_correctness(questions[counter], chatHistory[counter0 + index + j+1]['content']) == '1'

IndexError: ignored

In [ ]:
questions[counter],chatHistory[counter0 + index + j+1]['content']

In [ ]:
check_answer_correctness(questions[counter], chatHistory[counter0 + index + j+1]['content'])

In [ ]:
chatHistory,j,index,counter0,j+index+counter0,chatHistory[j+index+counter0+1],List


In [ ]:
edit_mode

In [ ]:
questions[counter-1]

In [ ]:
check_answer_correctness(questions[counter], chatHistory[counter0 + index+j+1]['content'])

In [ ]:
questions[counter-1]

In [ ]:
chatHistory[counter0 + index+j+1]['content']

In [ ]:
counter

In [ ]:
def List_collect(chatHistory,counter,index):
   List.append(chatHistory[counter+index]['content'])
   return List[1:-1]

In [ ]:
List,counter,index,List[counter-1],questions[counter-1]

In [ ]:
 List